## Subsetting Code:

In [1]:
#%pylab notebook
import os
import sys
import glob
from nco import Nco
import ease2conv as e2
from nco import custom as c
nco = Nco()

# Functions:

In [ ]:
def get_xy(ll_ul, ll_lr):
    N6 = e2.Ease2Transform("EASE2_N6.25km")
    N3 = e2.Ease2Transform("EASE2_N3.125km")
    # get x,y for 6.25
    row, col = N6.geographic_to_grid(ll_ul[0], ll_ul[1])
    x6ul, y6ul = N6.grid_to_map(row,col)
    row, col = N6.geographic_to_grid(ll_lr[0], ll_lr[1])
    x6lr, y6lr = N6.grid_to_map(row, col)
    # get x,y for 3.125
    row, col = N6.geographic_to_grid(ll_ul[0], ll_ul[1])
    x3ul, y3ul = N6.grid_to_map(row, col)
    row, col = N6.geographic_to_grid(ll_lr[0], ll_lr[1])
    x3lr, y3lr = N6.grid_to_map(row, col)
    list_6 = [x6ul, y6ul, x6lr, y6lr]
    list_3 = [x3ul, y3ul, x3lr, y3lr]
    return list_3, list_6

In [ ]:
def subset(list19, list37, list3, list6, path19, path37, wget):
    # for loop to take each file in the list and subset it's location 
    for file in list19:
        outfile = path19 + file
        infile = wget + file
        opt = [
            "-d x,%f,%f" % (list6[0],list6[2]),
            "-d y,%f,%f" % (list6[3],list6[1]),
            "-v TB"
        ]
        nco.ncks(input=infile, output=outfile, options=opt)
        
        print("Next in : %s" % file)
        print("Next out: %s" % outfile)
        # print("next unl out: %s" % unlfile)
        #subprocess.call(shlex.split(cmd), shell = False)
        os.remove(infile)
        print("Done")


    for file in list37: 
        outfile = path37 + file 
        infile = wget + file
        opt = [
            "-d x,%f,%f" % (list6[0],list6[2]),
            "-d y,%f,%f" % (list6[3],list6[1]),
            "-v TB"
        ]
        nco.ncks(input=infile, output=outfile, options=opt)
        
        print("Next in: %s" % infile)
        print("Next out: %s" % outfile)
        os.remove(infile)
        print("Done")  

In [ ]:
def concatenate(path, outfile_19, outfile_37):
    os.chdir(path)
    os.chdir(path + '/data' + '/Subsetted_19H')
    list19 = glob.glob('NSIDC*nc')
    list19.sort()
    # Concatenate 19GHz files:
    if len(list19) != 0:
        nco.ncrcat(input=list19, output = outfile_19, options=["-O"])
    else:
        print("No 19Ghz Files to Concatenate")

    # Concatenate 37GHz files: 
    os.chdir(path + '/data' + '/Subsetted_37H')
    list37 = glob.glob('NSIDC*nc')
    list37.sort()
    if len(list37) != 0:
        nco.ncrcat(input = list37, output = outfile_37, options = ["-O"])
    else: 
        print("No 37Ghz Files to Concatenate")

## Setup

In [2]:
path = os.getcwd()
path

'/Users/williamnorris/Team_GISProg'

## File Setup:

In [3]:
wget = path + "/data/wget/"
path19 = path + "/data/Subsetted_19H/"
path37 = path + "/data/Subsetted_37H/"


if not os.path.exists(wget):\
    os.makedirs(wget)
if not os.path.exists(path19):
    os.makedirs(path19)
if not os.path.exists(path37):
    os.makedirs(path37)

## Input Study Area

In [5]:
# upper left 
lat_lon_ul = [62.27, -140.17]

# lower right 
lat_lon_lr =[73.64, -166.08]

list_3, list_6 = get_xy(lat_lon_ul, lat_lon_lr)

In [6]:
os.chdir(wget)
# Make a list of the files to concatenate together for 19H
list19 = glob.glob("*19H-M-SIR*")
# Make list for the 37GHz
list37 = glob.glob("*37H-M-SIR*")
list19.sort()
list37.sort()

In [8]:
subset(list19, list37, list_3, list_6, path19, path37, wget)

In [9]:
outfile_19 = 'all_days_19H.nc'
outfile_37 = 'all_days_37H.nc'

In [15]:
concatenate(path, outfile_19, outfile_37)

No 19Ghz Files to Concatenate
